<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Test-runs" data-toc-modified-id="Test-runs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Test runs</a></span></li></ul></div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.cell_models import cell_v1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Test runs

In [ ]:
cell = cell_v1(division="amount")

In [ ]:
cell.params

In [ ]:
T = 5000

In [ ]:
for _ in range(T):
    cell.grow()

In [ ]:
plt.figure(figsize=(10, 6))
t_vec = np.arange(T+1)

plt.plot(t_vec, np.array(cell.M_hist)/cell.M_hist[0], label="M")
plt.plot(t_vec, np.array(cell.RB_hist)/cell.RB_hist[0], label="RB")
plt.plot(t_vec, np.array(cell.RB_c_hist)/cell.RB_c_hist[0], label="[RB]")

plt.yscale('log')
phase_vec = np.array([int(phase=="G1") for phase in cell.phase_hist])
x_min = 0
x_max = len(phase_vec)

phase=0
for k in range(len(phase_vec)):
    if (phase_vec[k]==1) and phase==0:
        x_min = k
        phase=1
    elif (phase_vec[k]==0) and phase==1:
        x_max=k-1
        phase=0
        plt.axvspan(x_min, x_max, color = 'lightgray', alpha=.3)
        

plt.axhline(cell.RB_transition/cell.RB_c_hist[0], color="red")
plt.axhline(cell.RB_division/cell.RB_c_hist[0], color="gray")
plt.legend()

Notes: 
- it looks very sensitive to the growth rate. Not sure there is a limit cycle in general to this problem. Can you sketch it? 
- maybe we need an RB amount threshold for the division? 
- a quick look at the RB amount threshold seems to indicate that it can yield limit cycles... ? 